In [1]:
import ee
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display
import json
import sys
sys.path.append('../Setup')  # Go up one level to reach Setup folder
from gee_setup import *

setup_gee()

✅ earthengine-api imported successfully
✅ folium imported successfully
✅ pandas imported successfully
✅ numpy imported successfully
✅ matplotlib imported successfully
✅ seaborn imported successfully
✅ IPython display imported successfully
✅ json imported successfully (built-in)
GEE utilities loaded successfully!
Use quick_setup() for one-line initialization
✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3


True

In [2]:
# Import analysis functions
sys.path.append('../functions')
from geolib import (
    count_images_by_year, 
    image_count, 
    count_images_by_week,
    mask_clouds_landsat,
    ndsi_l5,
    ndsi_l9
)

✅ Matplotlib configured
Google Earth Engine initialized successfully!
EE version: 1.6.3


In [3]:
# Fix the date ranges and add missing cloud filter
aoi = ee.Geometry.Polygon([[
    [-106.99065650552905, 39.088354116315266],
    [-106.71050513834155, 38.652122067998306], 
    [-106.14470923990405, 38.596331654032184],
    [-106.40838111490405, 39.21189167999414],
    [-106.99065650552905, 39.088354116315266]
]])

print("AOI defined - High Rocky Mountains polygon")
print(f"AOI area: {aoi.area().divide(1000000).getInfo():.2f} square kilometers")

# Fixed date ranges (remove square brackets)
date_range_1 = ee.DateRange('1990-01-01', '2000-01-01')  # Landsat 5 period
date_range_2 = ee.DateRange('2015-01-01', '2025-01-01')  # Landsat 8/9 period

# Create collections
landsat_1 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterDate(date_range_1) \
    .filterBounds(aoi) \
    .filter(ee.Filter.calendarRange(6, 6, 'month')) \
    .filter(ee.Filter.lt('CLOUD_COVER', 10))

landsat_2 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate(date_range_2) \
    .filterBounds(aoi) \
    .filter(ee.Filter.calendarRange(6, 6, 'month')) \
    .filter(ee.Filter.lt('CLOUD_COVER', 10))

test = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate(ee.DateRange('1980-01-01', '2025-01-01')) \
    .filterBounds(aoi) \
    .filter(ee.Filter.calendarRange(6, 6, 'month')) \
    .filter(ee.Filter.lt('CLOUD_COVER', 10))

collection = landsat_1.merge(landsat_2)

print("✅ Collections created successfully")

AOI defined - High Rocky Mountains polygon
AOI area: 2994.93 square kilometers
✅ Collections created successfully


In [4]:
# Test image counting functions
print("=== JUNE IMAGES BY YEAR ===")
year_results = count_images_by_year(landsat_2).getInfo()

# Display results in a clean format
for feature in year_results['features']:
    year = feature['properties']['year']
    count = feature['properties']['image_count']
    if count > 0:
        print(f"{year}: {count} images")

print(f"\n=== SUMMARY ===")
total_images = test.size().getInfo()
print(f"Total June images (all years): {total_images}")
print("✅ Function imports working correctly")

=== JUNE IMAGES BY YEAR ===
2015: 3 images
2016: 1 images
2017: 2 images
2018: 2 images
2022: 1 images
2023: 1 images
2024: 1 images

=== SUMMARY ===
Total June images (all years): 13
✅ Function imports working correctly


In [5]:
# Direct test - manually check specific years
print("=== MANUAL YEAR CHECK FOR LANDSAT_1 ===")

# Test specific years where we know there should be data
test_years = [1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999]

for year in test_years:
    # Create year-specific collection
    year_start = ee.Date.fromYMD(year, 1, 1)
    year_end = year_start.advance(1, 'year')
    
    year_collection = landsat_1.filterDate(year_start, year_end)
    count = year_collection.size().getInfo()
    
    if count > 0:
        print(f"{year}: {count} images")

print(f"\n=== VERIFICATION: Check if any images exist at all ===")
if landsat_1.size().getInfo() > 0:
    # Get first image info
    first = landsat_1.first()
    first_date = ee.Date(first.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    print(f"First image date: {first_date}")
    
    # Get date range of all images
    dates = landsat_1.aggregate_array('system:time_start')
    date_list = dates.getInfo()
    if date_list:
        import datetime
        readable_dates = []
        for timestamp in date_list[:5]:  # Show first 5 dates
            dt = datetime.datetime.fromtimestamp(timestamp/1000)
            readable_dates.append(dt.strftime('%Y-%m-%d'))
        print(f"Sample dates: {readable_dates}")

print("✅ Manual verification complete")

=== MANUAL YEAR CHECK FOR LANDSAT_1 ===
1990: 2 images
1991: 3 images
1992: 1 images
1993: 3 images
1994: 3 images
1995: 2 images
1996: 1 images
1997: 1 images
1998: 1 images
1999: 2 images

=== VERIFICATION: Check if any images exist at all ===
First image date: 1991-06-17
Sample dates: ['1991-06-17', '1992-06-19', '1993-06-22', '1994-06-09', '1994-06-25']
✅ Manual verification complete


In [6]:
## create cloud mask on each image
landsat_1_masked = landsat_1.map(mask_clouds_landsat)
landsat_2_masked = landsat_2.map(mask_clouds_landsat)


In [7]:
## apply ndvi to each image
landsat_1_ndsi = landsat_1_masked.map(ndsi_l5)
landsat_2_ndsi = landsat_2_masked.map(ndsi_l9)

In [8]:
## create a weighted average by year composite image from each collection


def yearly_composite(year, collection=collection):
    year_images = collection.filter(ee.Filter.calendarRange(year, year, 'year'))
    count = year_images.size()
    composite = year_images.mean()
    print(f"Year: {year}, Image Count: {count.getInfo()}")
    return composite.set('year', year).set('image_count', count)

def create_weighted_average(collection) -> ee.Image:
    years = collection.aggregate_array('system:time_start') \
    .map(lambda date: ee.Date(date) \
    .get('year')).distinct().sort()

    ## list of yearly_composite objects
    yearly_composites = []
    for year in years.getInfo():
        yearly_composites.append(yearly_composite(year, collection))

    ## convert list back into image collection
    composite_collection = ee.ImageCollection.fromImages(yearly_composites)

    ## average the yearly composites
    return composite_collection.mean()
    



In [9]:
landsat_1_yearly_avg = create_weighted_average(landsat_1_ndsi)
landsat_2_yearly_avg = create_weighted_average(landsat_2_ndsi)

difference_image = landsat_2_yearly_avg.subtract(landsat_1_yearly_avg)  

Year: 1990, Image Count: 2
Year: 1991, Image Count: 3
Year: 1992, Image Count: 1
Year: 1993, Image Count: 3
Year: 1994, Image Count: 3
Year: 1995, Image Count: 2
Year: 1996, Image Count: 1
Year: 1997, Image Count: 1
Year: 1998, Image Count: 1
Year: 1999, Image Count: 2
Year: 2015, Image Count: 3
Year: 2016, Image Count: 1
Year: 2017, Image Count: 2
Year: 2018, Image Count: 2
Year: 2022, Image Count: 1
Year: 2023, Image Count: 1
Year: 2024, Image Count: 1


In [10]:
import folium
aoi_center = aoi.centroid().coordinates().getInfo()[::-1]  # [lat, lon]
m = folium.Map(location=aoi_center, zoom_start=8)


In [ ]:
def add_ee_layer(folium_map, ee_image, vis_params, name, show=True):
    """
    Add an Earth Engine image to a Folium map
    
    Args:
        folium_map: Folium map object
        ee_image: Earth Engine image
        vis_params: Visualization parameters dict
        name: Layer name for the map
        show: Whether layer is visible by default
    """
    map_id_dict = ee_image.getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True,
        show=show
    ).add_to(folium_map)
    
    return folium_map

# Create your map
import folium
aoi_center = aoi.centroid().coordinates().getInfo()[::-1]  # [lat, lon]
m = folium.Map(location=aoi_center, zoom_start=8)

# Define visualization parameters
ndsi_vis = {
    'min': -0.5,
    'max': 1.0,
    #'palette': ['blue', 'cyan', 'white', 'red']
    'palette': ['red', 'yellow', 'green', 'blue', 'purple']
}

snow_vis = {
    'min': 0,
    'max': 1,
    'palette': ['brown', 'white']
}

# Add your layers easily
add_ee_layer(m, landsat_1_yearly_avg.select('NDSI'), ndsi_vis, 'Early Period NDSI (1990s)')
add_ee_layer(m, landsat_2_yearly_avg.select('NDSI'), ndsi_vis, 'Recent Period NDSI (2015+)')

# Add binary snow cover
early_snow = landsat_1_yearly_avg.select('NDSI').gt(0.4)
recent_snow = landsat_2_yearly_avg.select('NDSI').gt(0.4)

